# Polygon Triangulation Lab
## CS 480 Computational Geometry
### Prof. Bowers

__Objectives__

In this lab you will 

1. learn how to write a simple point set editor using the ipycanvas package, 
2. use the point set editor to draw an editable polygon, and
3. code a simple triangulation algorithm based on the algorithm that comes from the proof that all polygons have a triangulation.  

# Part 1: ipycanvas basics

We're first going to use `ipycanvas` to generate a point set that is editable. `ipycanvas` is a python package that allows a jupyter notebook like this one to use a web technology called HTML5 Canvas which allows you to draw directly into the browser. The documentation for ipycanvas can be found at [ipycanvas.readthedocs.io](https://ipycanvas.readthedocs.io). 

Creating a canvas to draw into is really easy. Here's code for drawing a few rectangles of various colors. 

In [ ]:
from ipycanvas import Canvas # Import the Canvas class

canvas = Canvas(size=(600, 600)) # Create a 600x600 pixel canvas

# Draw a few rectangles
canvas.fill_style = 'green'
canvas.fill_rect(50, 60, 20)

canvas.fill_style = 'red'
canvas.fill_rect(140, 30, 60, 70)

canvas.stroke_style = 'blue'
canvas.stroke_rect(30, 30, 40)

# To output the actual view of the canvas to Jupyter, we simply
# reference it on its own on the last line of the code block so 
# that Jupyter will show the result: 
canvas

You can also edit a canvas that is currently shown directly by calling more methods on it. You should already see your canvas above. Running the following code will add some blue rectangles to the existing canvas: 

In [ ]:
canvas.fill_style = 'blue'
canvas.fill_rects([30, 50, 100, 200], [250, 300, 345, 275], 15)

Modify the call to `fill_rects` in the block above to change the color to yellow and change the coordinates of the four squares drawn by a little bit. 

Notice that the calls to `fill_rects` just color over what has been already drawn. If you want to actually refresh the canvas before drawing, you need to call `canvas.clear()`. 

Notice also that `fill_rects` takes its rectangle corners as a list of x-coordinates followed by a list of y-coordinates. 

As a bit of a subtle point, every `.fill_*` or `.stroke_` command used as above generates communication between the presentation layer in the browser and the jupyter notebook server. Sending a lot of drawing commands one after another in this way is slow. `ipycanvas` has a special object called `hold_canvas` that can be used in conjunction with Pythons built in `with` construct to collect all of this into one communication dump, which vastly improves the drawing speed. Here's the same example as above except using a with construct. Note that we ammended the `import` statement to include `hold_canvas`. 

In [ ]:
from ipycanvas import Canvas, hold_canvas # Import the Canvas class

canvas2 = Canvas(size=(600, 600)) # Create a 600x600 pixel canvas

# Draw a few rectangles, but delay communication to the canvas until 
# all drawing is completed on the python side. 
with hold_canvas(canvas2):
    canvas2.fill_style = 'green'
    canvas2.fill_rect(50, 60, 20)

    canvas2.fill_style = 'red'
    canvas2.fill_rect(140, 30, 60, 70)

    canvas2.stroke_style = 'blue'
    canvas2.stroke_rect(30, 30, 40)

# To output the actual view of the canvas to Jupyter, we simply
# reference it on its own on the last line of the code block so 
# that Jupyter will show the result: 
canvas2

## Handling Mouse Events

Mouse events are handled by registering a function called a "handler" with the canvas for the mouse move (any time the mouse is moving over the canvas), mouse down (the mouse button is pressed), and mouse up (the mouse button is released) events. Each handler function should have two parameters `x` and `y` for the coordinates of the mouse pointer at the time the event fired. 

A quick and dirty architecture for basic user interaction with a canvas follows. The basic idea is to package up the drawing commands into a `draw()` function that first clears the canvas, and then draws the main scene. A global variable `mouseDown` tracks whether the mouse is currently up or down. In this dummy example, the color of the square changes depending on whether the mouse is up, down, or down and moving so that you can see how the interaction events are fired. The current color is stored in a global variable `squareColor` which is modified by the mouse handler functions `handle_mouse_down`, `handle_mouse_up`, and `handle_mouse_move`. 


In [ ]:
from ipycanvas import Canvas, hold_canvas

canvas3 = Canvas(size=(300, 300))
squareColor = 'black'
mouseDown = False 


def draw():
    global squareColor, canvas3, mouseDown
    with hold_canvas(canvas3):
        canvas3.clear()
        canvas3.fill_style = squareColor
        canvas3.fill_rect(0, 0, 300)
        if mouseDown:
            canvas3.stroke_style = 'white'
        else:
            canvas3.stroke_style = 'yellow'
        canvas3.stroke_rect(10, 10, 280)
        
def handle_mouse_down(x, y):
    global squareColor, mouseDown
    squareColor = 'red'
    mouseDown = True
    draw()

def handle_mouse_up(x, y):
    global squareColor, mouseDown
    squareColor = 'green'
    mouseDown = False
    draw()

def handle_mouse_move(x, y):
    global squareColor, mouseDown
    if mouseDown:
        squareColor = 'blue'
    else:
        squareColor = 'black'
    draw()

canvas3.on_mouse_move(handle_mouse_move)
canvas3.on_mouse_down(handle_mouse_down)
canvas3.on_mouse_up(handle_mouse_up)

draw() # Draw it to start off
canvas3

Here's a simple example where we keep track of the current `(x, y)` coordinates of the mouse cursor as a `PointE2` from the `koebe.py` library and draw a small square at that point.

In [ ]:
from koebe.geometries.euclidean2 import PointE2
from ipycanvas import Canvas, hold_canvas

canvas4 = Canvas(size=(300, 300))
point = PointE2(-10, -10)

def draw():
    global canvas4, point
    with hold_canvas(canvas4):
        canvas4.clear()
        canvas4.fill_style = 'red'
        canvas4.fill_rect(point.x - 8, point.y - 8, 16)

def handle_mouse_move(x, y):
    global point
    point = PointE2(x, y)
    draw()

canvas4.on_mouse_move(handle_mouse_move)

draw() # Draw it to start off
canvas4

# An editable point set. 

The code below creates an editable point set. Make sure you understand this code, because you are going to modify it. 

In [ ]:
from koebe.geometries.euclidean2 import PointE2
from ipycanvas import Canvas, hold_canvas

canvas5 = Canvas(size=(600,600))
points = [PointE2(100, 200), PointE2(200, 200), 
          PointE2(200, 300), PointE2(200, 400), 
          PointE2(300, 200), PointE2(300, 300), 
          PointE2(300, 400), PointE2(400, 400), 
          PointE2(500, 100)]
selectedPointIdx = -1

def draw():
    global canvas5, points, selectedPointIdx
    with hold_canvas(canvas5):
        canvas5.clear()
        canvas5.fill_style = 'blue'
        canvas5.fill_rects([int(p.x) - 4 for p in points], 
                           [int(p.y) - 4 for p in points], 
                           8)
        if selectedPointIdx != -1:
            canvas5.fill_style = 'red'
            canvas5.fill_rect(int(points[selectedPointIdx].x) - 4, 
                              int(points[selectedPointIdx].y) - 4, 
                              8)

def handle_mouse_down(x, y):
    global points, selectedPointIdx
    # See if any point is close to x, y
    cursorPoint = PointE2(x, y)
    sqDists = [p.distSqTo(cursorPoint) for p in points]
    minIdx = sqDists.index(min(sqDists))
    if sqDists[minIdx] < 24:
        selectedPointIdx = minIdx
    draw()

def handle_mouse_up(x, y):
    global selectedPointIdx
    selectedPointIdx = -1 # No point is selected anymore.
    draw()
    
def handle_mouse_move(x, y):
    global points, selectedPointIdx
    if selectedPointIdx >= 0:
        points[selectedPointIdx] = PointE2(x, y)
    draw()

canvas5.on_mouse_down(handle_mouse_down)
canvas5.on_mouse_up(handle_mouse_up)
canvas5.on_mouse_move(handle_mouse_move)
draw()
canvas5

# Storing a polygon

There are two natural ways to store a polygon. One is to store the vertices in order (generally counter-clockwise) in an array-based list. This is useful for applications where random access to the polygon's vertices is needed, but makes certain operations, like splitting a polygon along a diagonal more cumbersome. A second method is to store the polygon vertices as a doubly linked list. To do this we will define a container called a `Polygon` that keeps track of its `firstVertex`. Each vertex will be stored in a `Vertex` object that stores both its 2D point location (as a `PointE2` object accessible via the `.pos` attribute) and references to the previous (`.prev`) and next (`.next`) vertices in the polygon.

In [ ]:
class Vertex:
    def __init__(self, pos: PointE2):
        """
        Initializes the vertex at the given position.
        
        Attributes:
            pos: PointE2 - The position to initialize this vertex to.
        """
        self.pos = pos
        self.prev = None
        self.next = None
    
    def setNextVertex(self, nextVertex: "Vertex"):
        """
        Makes nextVertex the next vertex from this one. 
        """
        self.next = nextVertex
        nextVertex.prev = self

class Polygon:
    def __init__(self, vertexPositions = []):
        if len(vertexPositions) > 0:
            vertices = [Vertex(p) for p in vertexPositions]
            for i in range(len(vertices)):
                vertices[i - 1].setNextVertex(vertices[i])
            self.firstVertex = vertices[0]
        else:
            self.firstVertex = None
    
    def __iter__(self):
        """
        An iterator for the polygon that allows you to loop
        over a polygon's vertices using: 
        
        for vertex in polygon:
            ... do something with the vertex object ...
        """
        current = self.firstVertex
        while True:
            yield current
            current = current.next
            if current is self.firstVertex:
                break
    
    def vertexPositions(self):
        """
        Returns the positions of all ther vertices in order.
        """
        return [v.pos for v in self] # Uses the __iter__ to build a list
    
    def copy(self):
        """
        Returns a copy of this polygon. This performs a shallow copy
        since PointE2 objects are immutable. 
        """
        return Polygon(self.vertexPositions())
    
    def split(self, u, v):
        """
        Splits this polygon into two polygons. This is a destructive
        operation. After running this operation, the part of the polygon
        in order from u to v and then directly back to u is the stored
        in self, while the returned polygon is the other side of the split. 
        
        For example, if abcd is a polygon and we split with u = a and 
        v = c then the self polygon becomes abc and the returned polygon
        is cda. 
        
        Note that u and v must be vertices of this polygon. 
        """
        if u.next is v or v.next is u:
            return None # We can't split at an edge.
        
        result = Polygon()
        
        # Your solution here
        
        return result

# Your task

1. Complete the `split` method above for the `Polygon` class and use the code block below to test it. 

In [ ]:
# Here is a quick test of your split function: 

poly = Polygon([PointE2(0, 0), PointE2(100, 0), 
                PointE2(100, 100), PointE2(0, 100)])
A, B, C, D = tuple(poly) # Extract the vertices and name them.

# Print out the original polygon's coordinates.
print("Original polygon: ")
for v in poly:
    print(v.pos)

# Split the polygon between A and C
poly2 = poly.split(A, C)

# Should be the polygon ABC
print("\nAfter the split: ")
for v in poly:
    print(v.pos)

# Should be the polygon ACD
print("\nReturned polygon: ")
for v in poly2:
    print(v.pos)

2. Copy the editable point set canvas5 from the code block above to a code block below to begin working. 
3. Modify the code so that it edits the polygon object `poly` instead of the `points` list. 
4. Add code to the `draw()` method to draw the polygon (i.e. connect the dots--see the ipycanvas documentation, especially the "drawing shapes" section). 
5. Write a function called `inTriangle(p, a, b, c)`, which takes in four `PointE2` objects and returns true if and only if the point `p` is in the triangle `abc`. You may want to refer to lab 01 for the basic geometric primitives code and add a `leftHandTurn(a, b, c)` function. 
6. Write a function called `findEar` that takes as input a `Polygon` object and returns a `Vertex` object `v` of an ear `v.prev`, `v`, `v.next`.
7. Write a function called `triangulate` that takes as input a `Polygon` object and returns a list of `Polygon` objects representing the original polygon decomposed into triangles (hint: use your `findEar` method).  
8. Modify your drawing code so that it shows the triangulation as the points are edited. 